In [ ]:
"""
In the previous notebook, we covered what homomorphic encryption was, and an example of how we could use it
in Python 3.  This notebook turns that into a service, using the Flask web framework.
"""

# installing basic web server stuff
# pip install Flask Jinja2 MarkupSafe wsgiref

In [ ]:
# hello world server.py in Flask

import os, flask

app = flask.Flask(__name__)

@app.route('/')
def homepage():
    return 'hello world'

if __name__ == '__main__':
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port, debug=True)

In [ ]:
"""
That server doesn't do a whole lot.
For my goal, the geo example, we want to receive:
- encrypted latitude
- encrypted longitude
- the Paillier Public Key, which for some reason is two large numbers ('g' and 'n')

Here's how I might load the data:
"""

import os, flask, json
from flask import request
from phe import paillier

app = flask.Flask(__name__)

@app.route('/calculate')
def calculate_geo():
    # rebuild public key
    g = int(request.args.get('g'))
    n = int(request.args.get('n'))
    customerKey = paillier.PaillierPublicKey(g=g, n=n)

    # load the coordinates as EncryptedNumber type
    lat = paillier.EncryptedNumber(customerKey, int(request.args.get('lat')))
    lng = paillier.EncryptedNumber(customerKey, int(request.args.get('lng')))

    return 'data loaded'

if __name__ == '__main__':
    port = int(os.environ.get('PORT', 5000))
    app.run(host='0.0.0.0', port=port, debug=True)

In [ ]:
# let's continue writing calculate_geo to compare to the geo-box
# Paillier cryptosystem seems to works only on positive integers, so we cannot use -180.00 to +180.00 coordinates.
# Coordinates should be from 0 to 360, * 10^(decimal places of accuracy)... so I used 0 - 360,000

# include random library
from random import random

geobox = [west, south, east, north] # xmin, ymin, xmax, ymax order plz

def calculate_geo():
    g = int(request.args.get('g'))
    n = int(request.args.get('n'))
    customerKey = paillier.PaillierPublicKey(g=g, n=n)
    lat = paillier.EncryptedNumber(customerKey, int(request.args.get('lat')))
    lng = paillier.EncryptedNumber(customerKey, int(request.args.get('lng')))

    # use customer's public key to encrypt the geobox boundaries
    west = customerKey.encrypt(geobox[0])
    south = customerKey.encrypt(geobox[1])
    east = customerKey.encrypt(geobox[2])
    north = customerKey.encrypt(geobox[3])

    # calculate the offset of the point to the box bounds
    # inside the box, north and east should be positive, and south and west should be negative
    # (you can tell when a number overflows negative because it is many, many times greater than 0-360,000)
    # to make the box coordinates more secret, let's also multiply by random   
    southOffset = (south - lat) * random()
    northOffset = (north - lat) * random()
    westOffset = (west - lng) * random()
    eastOffset = (east - lng) * random()
    
    # the client, with the Private Key, will know how to decrypt and interpret this result
    return json.dumps({
      "south": str(northOffset.ciphertext()),
      "north": str(southOffset.ciphertext()),
      "west": str(westOffset.ciphertext()),
      "east": str(eastOffset.ciphertext()),
    })